In [59]:
import os

TITANIC_PATH = os.path.join("datasets", "titanic")

In [60]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [61]:
import pandas as pd
import numpy as np

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [62]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

# Посмотрим на данные

In [63]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Видно, что есть пропущенные переменные, и многие значения NA, возможно стоит задуматься об исключении некоторых признаков

In [64]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [65]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [66]:
train_data["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [67]:
train_data["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [68]:
train_data["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [69]:
train_data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

# Пайплайны для обработка данных

## Класс на выбор нужных признаков

In [70]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

## Генерация новых фичей

In [71]:
SibSp_ix, Parch_ix, Age_ix, Fare_ix = 1, 2, 0, 3

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_relate = True):
        self.add_relate = add_relate
    def fit(self, X, y=None):
        return self  
    def transform(self, X):
        age2 = (X[:,Age_ix])**2
        Fare2 = (X[:,Fare_ix])**2
        if self.add_relate:
            relate = X[:, Parch_ix] + X[:, SibSp_ix]
            return np.c_[X, age2, Fare2,
                         relate]
        else:
            return np.c_[X, age2, Fare2]

## Делаем объединённый пайплайн для численных признаков

In [72]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["Age", "SibSp", "Parch", "Fare"])),
    
        ("imputer", SimpleImputer(missing_values = np.nan,strategy="median")),
    ('attribs_adder',CombinedAttributesAdder(add_relate = False ))
    
    ])

In [73]:
num_pipeline.fit_transform(train_data)

array([[2.20000000e+01, 1.00000000e+00, 0.00000000e+00, 7.25000000e+00,
        4.84000000e+02, 5.25625000e+01],
       [3.80000000e+01, 1.00000000e+00, 0.00000000e+00, 7.12833000e+01,
        1.44400000e+03, 5.08130886e+03],
       [2.60000000e+01, 0.00000000e+00, 0.00000000e+00, 7.92500000e+00,
        6.76000000e+02, 6.28056250e+01],
       ...,
       [2.80000000e+01, 1.00000000e+00, 2.00000000e+00, 2.34500000e+01,
        7.84000000e+02, 5.49902500e+02],
       [2.60000000e+01, 0.00000000e+00, 0.00000000e+00, 3.00000000e+01,
        6.76000000e+02, 9.00000000e+02],
       [3.20000000e+01, 0.00000000e+00, 0.00000000e+00, 7.75000000e+00,
        1.02400000e+03, 6.00625000e+01]])

## Пайплайн для категориальных признаков

In [74]:
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [75]:
from sklearn.preprocessing import OneHotEncoder

In [76]:
cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["Pclass", "Sex", "Embarked"])),
        ("imputer", MostFrequentImputer()),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

In [77]:
cat_pipeline.fit_transform(train_data)

array([[0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

In [78]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [79]:
X_train = preprocess_pipeline.fit_transform(train_data)
X_train

array([[22.,  1.,  0., ...,  0.,  0.,  1.],
       [38.,  1.,  0., ...,  1.,  0.,  0.],
       [26.,  0.,  0., ...,  0.,  0.,  1.],
       ...,
       [28.,  1.,  2., ...,  0.,  0.,  1.],
       [26.,  0.,  0., ...,  1.,  0.,  0.],
       [32.,  0.,  0., ...,  0.,  1.,  0.]])

In [80]:
y_train = train_data["Survived"]

## Напишем простенький классификатор на основе RF

In [81]:
from sklearn.model_selection import cross_val_score

In [82]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

0.8194220292815798

In [83]:
from sklearn.model_selection import GridSearchCV

In [84]:
forest_clf = RandomForestClassifier()
param_grid = [
    {'n_estimators': [50,75,100,125,150], 'max_depth': [2, 4, 6, 8, 10,12,14,16]},  
  ]
gsc_rf = GridSearchCV(forest_clf, param_grid, cv=5,
                           scoring='accuracy',
                           return_train_score=True )

In [85]:
gsc_rf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [86]:
gsc_rf.best_params_

{'max_depth': 10, 'n_estimators': 150}

In [87]:
model = gsc_rf.best_estimator_

In [88]:
model_scores = cross_val_score(model, X_train, y_train, cv=10)
model_scores.mean()

0.8406951537850414

In [89]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [90]:
feature_importances = model.feature_importances_
feature_importances

array([0.1096809 , 0.03928189, 0.02841811, 0.12243434, 0.1099877 ,
       0.12146024, 0.0318695 , 0.01910888, 0.05387186, 0.17457689,
       0.15934682, 0.01138776, 0.00706418, 0.01151096])

## Предиктим и сохраняем результат

In [92]:
X_test = preprocess_pipeline.transform(test_data)
y_pred = model.predict(X_test)

## Был представлен самый базовый вариант решения данного соревнования(мой первый сабмит). В данном решение не ставилась цель получить лучший результат на лидерборде, или сделать прекрасный EDA, или обоснованный отбор и генерацию фичей. Просто сохраняю себе на память)